In [47]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [48]:
df = pd.read_excel('fake_job_postings.xlsx')
print(df.shape)
print(df.head())

(17880, 18)
   job_id                                      title            location  \
0       1                           Marketing Intern    US, NY, New York   
1       2  Customer Service - Cloud Video Production      NZ, , Auckland   
2       3    Commissioning Machinery Assistant (CMA)       US, IA, Wever   
3       4          Account Executive - Washington DC  US, DC, Washington   
4       5                        Bill Review Manager  US, FL, Fort Worth   

  department salary_range                                    company_profile  \
0  Marketing          NaN  We're Food52, and we've created a groundbreaki...   
1    Success          NaN  90 Seconds, the worlds Cloud Video Production ...   
2        NaN          NaN  Valor Services provides Workforce Solutions th...   
3      Sales          NaN  Our passion for improving quality of life thro...   
4        NaN          NaN  SpotSource Solutions LLC is a Global Human Cap...   

                                         descripti

In [49]:
# Dropping column job_id since dataframe automatically assigns an index
df.drop(['job_id'],axis=1,inplace=True)
print(df.shape)

(17880, 17)


##Cleaning description##

In [50]:
# Get the columns of company_profile, description, requirements, benefits and fraudulent into a new dataframe for easier manipulation
descdf = df[['company_profile','description','requirements','benefits','fraudulent']].copy()
print(descdf.shape)

(17880, 5)


In [51]:
print(descdf.head())

                                     company_profile  \
0  We're Food52, and we've created a groundbreaki...   
1  90 Seconds, the worlds Cloud Video Production ...   
2  Valor Services provides Workforce Solutions th...   
3  Our passion for improving quality of life thro...   
4  SpotSource Solutions LLC is a Global Human Cap...   

                                         description  \
0  Food52, a fast-growing, James Beard Award-winn...   
1  Organised - Focused - Vibrant - Awesome!Do you...   
2  Our client, located in Houston, is actively se...   
3  THE COMPANY: ESRI â€“ Environmental Systems Re...   
4  JOB TITLE: Itemization Review ManagerLOCATION:...   

                                        requirements  \
0  Experience with content management systems a m...   
1  What we expect from you:Your key responsibilit...   
2  Implement pre-commissioning and commissioning ...   
3  EDUCATION:Â Bachelorâ€™s or Masterâ€™s in GIS,...   
4  QUALIFICATIONS:RN license in the State of T

In [52]:
# Cleaning the description column first, the easiest since there are no missing values.
def cleanDescrip(desc):
  
  # First remove #URLasna209213..a# if present
  reg = "#URL_[a-z0-9]*#"
  res_str = re.sub(reg," ",desc)
  # print(res_str)
  
  # Replace â€™ with '
  res_str = re.sub(r"â€™","'",res_str)
  # print(res_str)

  # Next, remove non-alphanumeric and non-punctuation characters
  reg = "[^a-zA-Z0-9\s:.',-_!?()@;]"
  res_str = re.sub(reg,"",res_str)
  # print(res_str)

  # Now, correct the wordWord instances
  reg = "([a-z])([A-Z])"
  res_str = re.sub(reg,r"\1 \2",res_str)      #Referred to from here - https://stackoverflow.com/questions/38998547/insert-space-between-characters-regex
  # print(res_str)

  # Add a space where : is present, to ease later cleaning
  res_str = re.sub(":"," : ",res_str)

  # Replace multiple spaces with a single space, and remove spaces from ends
  reg = "[\s]+"
  res_str = re.sub(reg," ",res_str).strip()
  # print(res_str)

  # Now we can do the usual cleaning
  res_str = clean_text(res_str)
  # print(res_str)
  return res_str

In [53]:
# From HW 1
# References
# https://towardsdatascience.com/how-to-efficiently-remove-punctuations-from-a-string-899ad4a059fb
# https://datagy.io/python-remove-punctuation-from-string/
# https://riptutorial.com/nltk/example/27393/porter-stemmer

def clean_text(review):
  '''
  Input:
      review: a string containing a review.
  Output:
      review_cleaned: a processed review. 
  '''
  stop_words = set(stopwords.words('english'))
  
  review_cleaned = re.sub(r'[^\w\s]',"", review)         #Replace non alphanumeric and non-whitespace characters with empty character
  review_cleaned = review_cleaned.lower()                #Lowercase the entire review
  words = nltk.word_tokenize(review_cleaned)
  remStop = []
  for word in words:
    if word not in stop_words:
      remStop.append(word)                               #Convert review into a list, and then add the non-stopword words to a new list and make that into a string
  
  # ps = PorterStemmer()                    
  # for ind in range(len(remStop)):                        #Applying porter stemmer
  #   remStop[ind] = ps.stem(remStop[ind])
  
  review_cleaned = ' '.join(words for words in remStop)
  # review_cleaned = review_cleaned.strip()
  return review_cleaned

In [54]:
print(descdf['description'][9])

The Customer Service Associate will be based in Phoenix, AZ. The right candidate will be an integral part of our talented team, supporting our continued growth.Responsibilities:Perform various Mail Center activities (sorting, metering, folding, inserting, delivery, pickup, etc.)Lift heavy boxes, files or paper when neededMaintain the highest levels of customer care while demonstrating a friendly and cooperative attitudeDemonstrate flexibility in satisfying customer demands in a high volume, production environmentConsistently adhere to business procedure guidelinesAdhere to all safety proceduresTake direction from supervisor or site managerMaintain all logs and reporting documentation; attention to detailParticipate in cross-training and perform other duties as assigned (Filing, outgoing shipments, etc)Operating mailing, copy or scanning equipmentShipping &amp; ReceivingHandle time-sensitive material like confidential, urgent packagesPerform other tasks as assignedScanning incoming mail

In [55]:
tempdf = pd.DataFrame(columns=['description'])

In [56]:
# Applying this to entire description column in the df
# descdf['description'] = descdf['description'].apply(cleanDescrip)
# ls = [3030,3951,4184,7612,11893,12189,13529,16690,17513]    #These are rows which have literally #NAME? or N/A as description
ls = []
for i in range(descdf.shape[0]):
  # if i in ls:
  #   continue
  # tempdf.append(cleanDescrip(descdf['description'][i]))
  try:
    descdf['description'][i] = cleanDescrip(descdf['description'][i])
    # tempdf.append(pd.Series(cleanDescrip(descdf['description'][i])),ignore_index=True)
  except Exception as e:
    print(i)
    ls.append(i)
    print(e)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


3030
expected string or bytes-like object
3951
expected string or bytes-like object
4184
expected string or bytes-like object
7612
expected string or bytes-like object
11893
expected string or bytes-like object
12189
expected string or bytes-like object
13529
expected string or bytes-like object
16690
expected string or bytes-like object
17513
expected string or bytes-like object


In [57]:
# We can drop those bad rows, since the number of rows is large
for i in ls:
  descdf = descdf.drop(descdf.index[i])
# descdf = descdf.reset_index()

In [58]:
print(descdf.shape)

(17871, 5)


In [59]:
ls = []
print(ls)

[]


In [60]:
print(descdf.head())

                                     company_profile  \
0  We're Food52, and we've created a groundbreaki...   
1  90 Seconds, the worlds Cloud Video Production ...   
2  Valor Services provides Workforce Solutions th...   
3  Our passion for improving quality of life thro...   
4  SpotSource Solutions LLC is a Global Human Cap...   

                                         description  \
0  food52 fastgrowing james beard awardwinning on...   
1  organised focused vibrant awesomedo passion cu...   
2  client located houston actively seeking experi...   
3  company esri environmental systems research in...   
4  job title itemization review manager location ...   

                                        requirements  \
0  Experience with content management systems a m...   
1  What we expect from you:Your key responsibilit...   
2  Implement pre-commissioning and commissioning ...   
3  EDUCATION:Â Bachelorâ€™s or Masterâ€™s in GIS,...   
4  QUALIFICATIONS:RN license in the State of T

##Cleaning company profile##

In [61]:
# description is now cleaned
# Onto company_profile
# Cannot simply remove the rows with no company profile because they form a significant chunk of fraudulent postings
# Have to do something about them
tempdf = pd.DataFrame(columns=['company_profile'])
tempdf = descdf[['company_profile']]

In [62]:
print(tempdf.shape)

(17871, 1)


In [63]:
print(tempdf.head())

                                     company_profile
0  We're Food52, and we've created a groundbreaki...
1  90 Seconds, the worlds Cloud Video Production ...
2  Valor Services provides Workforce Solutions th...
3  Our passion for improving quality of life thro...
4  SpotSource Solutions LLC is a Global Human Cap...


In [64]:
print(type(tempdf))

<class 'pandas.core.frame.DataFrame'>


In [65]:
tempdf.isnull()

,company_profile
0,False
1,False
2,False
3,False
4,False
...,...
17875,False
17876,False
17877,False
17878,True


In [66]:
type(tempdf.isnull())

pandas.core.frame.DataFrame

In [67]:
count = 0
for i in tempdf.isnull()['company_profile']:
  if i:
    count += 1
print(count)

3306


In [68]:
temp2 = tempdf.copy()

In [69]:
temp2.fillna("NA",inplace=True)

In [70]:
print(temp2.shape)
count = 0
for i in temp2.isnull()['company_profile']:
  if i:
    count += 1
print(count)

(17871, 1)
0


In [71]:
def cleanCols(comp):
  if comp == "NA":
    return comp
  res_str = cleanDescrip(comp)
  return res_str

In [72]:
for i in range(len(temp2['company_profile'])):
  try:
    descdf['company_profile'][i] = cleanCols(temp2['company_profile'][i])
  except Exception as e:
    print(i)
    print(e)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


3030
3030
3952
3952
4186
4186
7615
7615
11897
11897
12194
12194
13535
13535
16697
16697
17521
17521


In [73]:
print(descdf.head())

                                     company_profile  \
0  food52 weve created groundbreaking awardwinnin...   
1  90 seconds worlds cloud video production servi...   
2  valor services provides workforce solutions me...   
3  passion improving quality life geography heart...   
4  spot source solutions llc global human capital...   

                                         description  \
0  food52 fastgrowing james beard awardwinning on...   
1  organised focused vibrant awesomedo passion cu...   
2  client located houston actively seeking experi...   
3  company esri environmental systems research in...   
4  job title itemization review manager location ...   

                                        requirements  \
0  Experience with content management systems a m...   
1  What we expect from you:Your key responsibilit...   
2  Implement pre-commissioning and commissioning ...   
3  EDUCATION:Â Bachelorâ€™s or Masterâ€™s in GIS,...   
4  QUALIFICATIONS:RN license in the State of T

##Benefits and requirements column cleaning##

In [74]:
# Do same stuff as company profile to requirements and benefits
tempdf = pd.DataFrame(columns=['requirements','benefits'])
tempdf = descdf[['requirements','benefits']].copy()

In [75]:
print(tempdf.shape)

(17871, 2)


In [76]:
tempdf.isnull()

,requirements,benefits
0,False,True
1,False,False
2,False,True
3,False,False
4,False,False
...,...,...
17875,False,False
17876,False,False
17877,False,True
17878,False,False


In [77]:
reqcount = 0
bencount = 0
for i in tempdf.isnull()['requirements']:
  if i:
    reqcount += 1
for i in tempdf.isnull()['benefits']:
  if i:
    bencount += 1
print("Reqcount - ", str(reqcount))
print("Bencount - ", str(bencount))

Reqcount -  2732
Bencount -  7236


In [78]:
tempdf.fillna("NA",inplace=True)    # Fill NA where data is missing
reqcount = 0
bencount = 0
for i in tempdf.isnull()['requirements']:
  if i:
    reqcount += 1
for i in tempdf.isnull()['benefits']:
  if i:
    bencount += 1
print("Reqcount - ", str(reqcount))
print("Bencount - ", str(bencount))

Reqcount -  0
Bencount -  0


In [79]:
len(tempdf['requirements'])

17871

In [80]:
tempdf.head()

,requirements,benefits
0,Experience with content management systems a m...,NA
1,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...
2,Implement pre-commissioning and commissioning ...,NA
3,"EDUCATION:Â Bachelorâ€™s or Masterâ€™s in GIS,...",Our culture is anything but corporateâ€”we hav...
4,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered


In [81]:
for i in range(len(tempdf['requirements'])):
  try:
    descdf['requirements'][i] = cleanCols(tempdf['requirements'][i])
    descdf['benefits'][i] = cleanCols(tempdf['requirements'][i])
  except Exception as e:
    print(i)
    print(e)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


3030
3030
3952
3952
4186
4186
7615
7615
11897
11897
12194
12194
13535
13535
16697
16697
17521
17521


In [82]:
print(descdf.head())

                                     company_profile  \
0  food52 weve created groundbreaking awardwinnin...   
1  90 seconds worlds cloud video production servi...   
2  valor services provides workforce solutions me...   
3  passion improving quality life geography heart...   
4  spot source solutions llc global human capital...   

                                         description  \
0  food52 fastgrowing james beard awardwinning on...   
1  organised focused vibrant awesomedo passion cu...   
2  client located houston actively seeking experi...   
3  company esri environmental systems research in...   
4  job title itemization review manager location ...   

                                        requirements  \
0  experience content management systems major pl...   
1  expect key responsibility communicate client 9...   
2  implement precommissioning commissioning proce...   
3  education bachelors masters gis business admin...   
4  qualifications rn license state texas diplo

In [84]:
finaldf = descdf.copy()

In [85]:
print(finaldf.shape)

(17871, 5)


In [86]:
finaldf.reset_index(inplace=True)

In [87]:
finaldf.shape

(17871, 6)

In [88]:
finaldf.drop("index",axis=1,inplace=True)

In [92]:
# Storing the cleaned df into a pickle file for easy sharing
finaldf.to_pickle("cleaned df.pkl")

In [90]:
finaldf

,company_profile,description,requirements,benefits,fraudulent
0,food52 weve created groundbreaking awardwinnin...,food52 fastgrowing james beard awardwinning on...,experience content management systems major pl...,experience content management systems major pl...,0
1,90 seconds worlds cloud video production servi...,organised focused vibrant awesomedo passion cu...,expect key responsibility communicate client 9...,expect key responsibility communicate client 9...,0
2,valor services provides workforce solutions me...,client located houston actively seeking experi...,implement precommissioning commissioning proce...,implement precommissioning commissioning proce...,0
3,passion improving quality life geography heart...,company esri environmental systems research in...,education bachelors masters gis business admin...,education bachelors masters gis business admin...,0
4,spot source solutions llc global human capital...,job title itemization review manager location ...,qualifications rn license state texas diploma ...,qualifications rn license state texas diploma ...,0
...,...,...,...,...,...
17866,Vend is looking for some awesome new talent to...,case first time youve visited website vend awa...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0
17867,WebLinc is the e-commerce platform and service...,payroll accountant focus primarily payroll fun...,- B.A. or B.S. in Accounting-Â Desire to have ...,Health &amp; WellnessMedical planPrescription ...,0
17868,We Provide Full Time Permanent Positions for m...,experienced project cost control staff enginee...,At least 12 years professional experience.Abil...,NaN,0
17869,NaN,nemsia studios looking experienced visualgraph...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0
